## Objective

After looking at my handy map, I see outliers per cap bc the pop is low. I know the Census tries to get its Census blocks to be of about the same pop. Is there a clean cutoff to throw out the low-pop block groups? Would there be any useful signal in each of them?

In [46]:
from __future__ import division
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline
import json
import plotly.plotly as py
import cufflinks as cf

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")
rcParams['figure.figsize'] = 20,5

In [2]:
cd ../preprocessing

/media/aok1425/1f5fa7c1-6532-4a01-8076-91f01ca10cae/PycharmProjects/galvanize/311-prediction-times/preprocessing


In [3]:
from transform_for_num_issues_pred import add_population

In [12]:
cd ../app/

/media/aok1425/1f5fa7c1-6532-4a01-8076-91f01ca10cae/PycharmProjects/galvanize/311-prediction-times/app


In [37]:
from models import make_q1_json, BLOCK_GROUP_BLACKLIST

ImportError: cannot import name BLOCK_GROUP_BLACKLIST

In [4]:
d = add_population(df=None, just_dict=True)

In [14]:
data = make_q1_json()

## Let's check

In [67]:
df = pd.DataFrame([d, data[u'top_n_all_yrs_totals']]).T
df.drop(BLOCK_GROUP_BLACKLIST, inplace=True)
df.columns = ['population', 'num_issues']
df['num_issues_per_1000'] = df.num_issues * 1000 / df.population
df['keep'] = True
print df.shape
df.head()

(641, 4)


,population,num_issues,num_issues_per_1000,keep
0001001,1374.0,1673.0,1217.612809,True
0001002,1042.0,813.0,780.230326,True
0001003,498.0,719.0,1443.775100,True
0001004,757.0,570.0,752.972259,True
0002011,985.0,881.0,894.416244,True


In [27]:
df.population.describe()

count     646.000000
mean     1174.797214
std       559.851811
min         0.000000
25%       802.500000
50%      1078.500000
75%      1467.750000
max      3841.000000
Name: population, dtype: float64

In [56]:
# rcParams['figure.figsize'] = 20,10
# sns.regplot(x='population', y='num_issues_per_1000', data=df, fit_reg=False);

In [55]:
df.reset_index().iplot(kind='scatter', mode='markers', text='index', x='population', y='num_issues_per_1000', size=5)

The points on the top-left quadrant, and also the bottom-right look like outliers. Specifically, the top-left ones are what I was concerned about: the block groups with low pops so as to inflate the number of issues per 1000.

I could use K-means with k=3 to choose the outliers, but a simple visual cutoff and manual inspection will work too. I want to investigate the points where num_issues_per_1000 > 2000, and to a lesser degree, where population > 2750.

## Manual investigation

For y > 2000:

| id      | residential? | comments                        |
|---------|--------------|---------------------------------|
| 0102034  | FALSE        | fenway                          |
| 0107013  | FALSE        | hynes convention ctr            |
| 107021  | TRUE         | copley                          |
| 201011  | TRUE         | beacon hill                     |
| 404012  | TRUE         | Charlestown; over the bridge    |
| 0512001  | FALSE        | east boston waterfront          |
| 0612002  | FALSE        | tracks west of Andrew           |
| 701011  | TRUE         | South Station-ish               |
| 0701012  | FALSE        | downtown crossing               |
| 703002  | TRUE         | by Chinatown and I-90           |
| 806013  | TRUE         | Dudley                          |
| 1101033 | FALSE        | btwn arboretum and forest hills |
| 9812021 | FALSE        | cruise SB place                 |

For x > 2750:

| id     | residential? | comments                              |
|--------|--------------|---------------------------------------|
| 0005024| FALSE        | BC                                    |
| 0008032| FALSE        | BU western part                       |
| 0103002| FALSE        | Simmons, Fenway                       |
| 0104051| FALSE        | Northeastern                          |
| 0507001| TRUE         | eastie                                |
| 0702003| TRUE         | emerson/chinatown                     |
| 0806011| TRUE         | orange line by mass ave station to NE |

In [72]:
OUTLIERS_COMMERCIAL_INDUSTRIAL = ['0102034', '0107013', '0512001', '0612002', '0701012', '1101033', '9812021']
OUTLIERS_LOW_POP = ['0005024', '0008032', '0103002', '0104051']

## Conclusion

Since the same 5 blocks groups show up year after year as having higher density than the others, and bc that's bc they're in low-population areas, they're outliers. Let's remove them.